## import, plk 파일 확인

In [1]:
import pickle
import numpy as np
import os


## 모델에 필요한거
import numpy as np
from common.layers import Convolution, Affine, Relu, BatchNormalization
from common.functions import softmax, cross_entropy_error
from common.util import im2col, col2im

In [2]:
# .pkl 구조 확인
import pickle

with open("Resnet-20_epoch30_model.pkl", "rb") as f:
    data = pickle.load(f)

# 1. 최상위 타입 및 키
print(type(data))           # dict expected
print(data.keys())          # ['model', 'optimizer']

# 2. 모델 파라미터 키 목록
model = data['model']
print(type(model))          # dict expected
print(list(model.keys())[:10])  # ['fc_W', 'fc_b', '0_W', ...]

# 3. 파라미터 shape 확인
for k in model.keys():
    print(f"{k}: {model[k].shape}")



<class 'dict'>
dict_keys(['model', 'optimizer'])
<class 'dict'>
['fc_W', 'fc_b', '0_W', '0_b', '1_W', '1_b', '2_W', '2_b', '3_W', '3_b']
fc_W: (64, 100)
fc_b: (100,)
0_W: (16, 16, 3, 3)
0_b: (16,)
1_W: (16, 16, 3, 3)
1_b: (16,)
2_W: (16, 16, 3, 3)
2_b: (16,)
3_W: (16, 16, 3, 3)
3_b: (16,)
4_W: (16, 16, 3, 3)
4_b: (16,)
5_W: (16, 16, 3, 3)
5_b: (16,)
6_W: (32, 16, 3, 3)
6_b: (32,)
7_W: (32, 32, 3, 3)
7_b: (32,)
8_W: (32, 16, 1, 1)
8_b: (32,)
9_W: (32, 32, 3, 3)
9_b: (32,)
10_W: (32, 32, 3, 3)
10_b: (32,)
11_W: (32, 32, 3, 3)
11_b: (32,)
12_W: (32, 32, 3, 3)
12_b: (32,)
13_W: (64, 32, 3, 3)
13_b: (64,)
14_W: (64, 64, 3, 3)
14_b: (64,)
15_W: (64, 32, 1, 1)
15_b: (64,)
16_W: (64, 64, 3, 3)
16_b: (64,)
17_W: (64, 64, 3, 3)
17_b: (64,)
18_W: (64, 64, 3, 3)
18_b: (64,)
19_W: (64, 64, 3, 3)
19_b: (64,)


In [3]:
with open('Resnet-20_epoch30_model.pkl', 'rb') as f:
    data = pickle.load(f)

print(type(data))         # dict여야 함
print(data.keys())        # model, optimizer
print(data['model'].keys())  # 0_W ~ 19_W, fc_W, fc_b 있어야 함


<class 'dict'>
dict_keys(['model', 'optimizer'])
dict_keys(['fc_W', 'fc_b', '0_W', '0_b', '1_W', '1_b', '2_W', '2_b', '3_W', '3_b', '4_W', '4_b', '5_W', '5_b', '6_W', '6_b', '7_W', '7_b', '8_W', '8_b', '9_W', '9_b', '10_W', '10_b', '11_W', '11_b', '12_W', '12_b', '13_W', '13_b', '14_W', '14_b', '15_W', '15_b', '16_W', '16_b', '17_W', '17_b', '18_W', '18_b', '19_W', '19_b'])


## 데이터셋 불러오기

In [4]:
# 데이터셋 로드 및 전처리
import numpy as np
import pickle
import os
import urllib.request
import tarfile

def download_cifar100(dest="./cifar-100-python"):
    url = "https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz"
    filename = "cifar-100-python.tar.gz"
    
    def is_within_directory(directory, target):
        abs_directory = os.path.abspath(directory)
        abs_target = os.path.abspath(target)
        return os.path.commonprefix([abs_directory, abs_target]) == abs_directory

    def safe_extract(tar, path=".", members=None):
        for member in tar.getmembers():
            member_path = os.path.join(path, member.name)
            if not is_within_directory(path, member_path):
                raise Exception("Attempted Path Traversal in Tar File")
        tar.extractall(path, members)

    if not os.path.exists(dest):
        os.makedirs(dest, exist_ok=True)
        urllib.request.urlretrieve(url, filename)
        with tarfile.open(filename, "r:gz") as tar:
            safe_extract(tar, path="./")
        print("CIFAR-100 downloaded and extracted.")
    else:
        print("CIFAR-100 already downloaded.")

def load_cifar100(data_dir="./cifar-100-python"):
    def load_batch(filename):
        with open(filename, 'rb') as f:
            dict = pickle.load(f, encoding='bytes')
            data = dict[b'data']
            labels = dict[b'fine_labels']
            coarse_labels = dict[b'coarse_labels']
            return data, labels, coarse_labels

    x_train, y_train, y_train_coarse = load_batch(os.path.join(data_dir, "train"))
    x_test, y_test, y_test_coarse = load_batch(os.path.join(data_dir, "test"))

    x_train = x_train.reshape(-1, 3, 32, 32).astype(np.float32) / 255.0
    x_test = x_test.reshape(-1, 3, 32, 32).astype(np.float32) / 255.0

    y_train = np.array(y_train)
    y_test = np.array(y_test)
    y_train_coarse = np.array(y_train_coarse)
    y_test_coarse = np.array(y_test_coarse)

    val_size = int(0.1 * len(x_train))
    x_val = x_train[:val_size]
    y_val = y_train[:val_size]
    x_train = x_train[val_size:]
    y_train = y_train[val_size:]

    return (x_train, y_train), (x_val, y_val), (x_test, y_test), (y_train_coarse, y_test_coarse) # 지금 y_valid_coarse가 없는거지?

In [5]:
# 데이터 다운로드 및 로딩
download_cifar100()
(x_train, y_train), (x_val, y_val), (x_test, y_test), (y_train_coarse, y_test_coarse) = load_cifar100()

# 데이터셋 정보 출력
print(" CIFAR-100 Dataset Loaded!")
print(f"Train X shape: {x_train.shape}, Train Y shape: {y_train.shape}")
print(f"Val   X shape: {x_val.shape}, Val   Y shape: {y_val.shape}")
print(f"Test  X shape: {x_test.shape}, Test  Y shape: {y_test.shape}")
print(f"Coarse Labels - Train: {y_train_coarse.shape}, Test: {y_test_coarse.shape}")

CIFAR-100 already downloaded.
 CIFAR-100 Dataset Loaded!
Train X shape: (45000, 3, 32, 32), Train Y shape: (45000,)
Val   X shape: (5000, 3, 32, 32), Val   Y shape: (5000,)
Test  X shape: (10000, 3, 32, 32), Test  Y shape: (10000,)
Coarse Labels - Train: (50000,), Test: (10000,)


In [6]:
# 데이터셋 변수명 정리를 해야하나?
## 노노 일단 test 에 대해서만 할거니까 
##나중에 필요하면 추가하기

## 서아 모델 구조

In [7]:
class ResidualBlock:
    def __init__(self, in_channels, out_channels, stride=1):
        self.stride = stride
        self.equal_in_out = (in_channels == out_channels and stride == 1)

        self.conv1 = Convolution(
            W=np.random.randn(out_channels, in_channels, 3, 3) * np.sqrt(2. / in_channels),
            b=np.zeros(out_channels),
            stride=stride,
            pad=1
        )
        self.bn1 = BatchNormalization(gamma=np.ones(out_channels), beta=np.zeros(out_channels))
        self.relu1 = Relu()

        self.conv2 = Convolution(
            W=np.random.randn(out_channels, out_channels, 3, 3) * np.sqrt(2. / out_channels),
            b=np.zeros(out_channels),
            stride=1,
            pad=1
        )
        self.bn2 = BatchNormalization(gamma=np.ones(out_channels), beta=np.zeros(out_channels))
        self.relu2 = Relu()

        if not self.equal_in_out:
            self.shortcut = Convolution(
                W=np.random.randn(out_channels, in_channels, 1, 1) * np.sqrt(2. / in_channels),
                b=np.zeros(out_channels),
                stride=stride,
                pad=0
            )
            self.bn_shortcut = BatchNormalization(gamma=np.ones(out_channels), beta=np.zeros(out_channels))

    def forward(self, x, train_flg=True):
        self.x = x

        out = self.conv1.forward(x)
        out = self.bn1.forward(out, train_flg)
        out = self.relu1.forward(out)

        out = self.conv2.forward(out)
        out = self.bn2.forward(out, train_flg)
        self.out_main = out

        if self.equal_in_out:
            shortcut = x
        else:
            shortcut = self.shortcut.forward(x)
            shortcut = self.bn_shortcut.forward(shortcut, train_flg)
        self.out_shortcut = shortcut

        out += shortcut
        out = self.relu2.forward(out)

        return out

    def backward(self, dout):
        dout = self.relu2.backward(dout)

        dshortcut = dout.copy()
        dmain = dout.copy()

        dmain = self.bn2.backward(dmain)
        dmain = self.conv2.backward(dmain)

        dmain = self.relu1.backward(dmain)
        dmain = self.bn1.backward(dmain)
        dmain = self.conv1.backward(dmain)

        if not self.equal_in_out:
            dshortcut = self.bn_shortcut.backward(dshortcut)
            dshortcut = self.shortcut.backward(dshortcut)

        dx = dmain + dshortcut
        return dx


class ResNet20:
    def __init__(self, input_dim=(3, 32, 32), num_classes=100):
        self.params = []
        self.trainable_layers = []

        self.conv1 = Convolution(
            W=np.random.randn(16, 3, 3, 3) * np.sqrt(2. / 3),
            b=np.zeros(16),
            stride=1,
            pad=1
        )
        self.bn1 = BatchNormalization(gamma=np.ones(16), beta=np.zeros(16))
        self.relu1 = Relu()

        self.layer1 = [ResidualBlock(16, 16, stride=1) for _ in range(3)]
        self.layer2 = [ResidualBlock(16 if i == 0 else 32, 32, stride=2 if i == 0 else 1) for i in range(3)]
        self.layer3 = [ResidualBlock(32 if i == 0 else 64, 64, stride=2 if i == 0 else 1) for i in range(3)]

        self.fc = Affine(W=np.random.randn(64, num_classes) * np.sqrt(2. / 64), b=np.zeros(num_classes))

    def forward(self, x, train_flg=True):
        self.input = x

        out = self.conv1.forward(x)
        out = self.bn1.forward(out, train_flg)
        out = self.relu1.forward(out)

        for block in self.layer1:
            out = block.forward(out, train_flg)
        for block in self.layer2:
            out = block.forward(out, train_flg)
        for block in self.layer3:
            out = block.forward(out, train_flg)

        self.feature_map = out

        N, C, H, W = out.shape
        out = out.mean(axis=(2, 3))

        self.pooled = out
        out = self.fc.forward(out)
        return out

    def predict(self, x, batch_size=100):
        y_list = []
        for i in range(0, x.shape[0], batch_size):
            x_batch = x[i:i+batch_size]
            y_batch = self.forward(x_batch, train_flg=False)
            y_list.append(y_batch)
        return np.concatenate(y_list, axis=0)

    def loss(self, x, t):
        y = self.forward(x, train_flg=True)
        return cross_entropy_error(softmax(y), t)

    def accuracy(self, x, t, batch_size=100):
        acc = 0.0
        total = x.shape[0]
        for i in range(0, total, batch_size):
            x_batch = x[i:i+batch_size]
            t_batch = t[i:i+batch_size]

            y = self.predict(x_batch)
            y = np.argmax(y, axis=1)

            if t.ndim != 1:
                t_batch = np.argmax(t_batch, axis=1)

            acc += np.sum(y == t_batch)

        return acc / total

    def backward(self, dout):
        dout = self.fc.backward(dout)
        dout = dout.reshape(self.feature_map.shape[0], self.feature_map.shape[1], 1, 1)
        dout = dout.repeat(self.feature_map.shape[2], axis=2).repeat(self.feature_map.shape[3], axis=3)

        for block in reversed(self.layer3):
            dout = block.backward(dout)
        for block in reversed(self.layer2):
            dout = block.backward(dout)
        for block in reversed(self.layer1):
            dout = block.backward(dout)

        dout = self.relu1.backward(dout)
        dout = self.bn1.backward(dout)
        dout = self.conv1.backward(dout)
        return dout

## weight, bias 값 불러오기

In [8]:
def load_weights(model, weights):
    idx = 0
    # layer1
    for i in range(3):
        block = model.layer1[i]
        block.conv1.W = weights[f'{idx}_W']
        block.conv1.b = weights[f'{idx}_b']
        idx += 1
        block.conv2.W = weights[f'{idx}_W']
        block.conv2.b = weights[f'{idx}_b']
        idx += 1

    # layer2
    for i in range(3):
        block = model.layer2[i]
        block.conv1.W = weights[f'{idx}_W']
        block.conv1.b = weights[f'{idx}_b']
        idx += 1
        block.conv2.W = weights[f'{idx}_W']
        block.conv2.b = weights[f'{idx}_b']
        idx += 1
        if not block.equal_in_out:
            block.shortcut.W = weights[f'{idx}_W']
            block.shortcut.b = weights[f'{idx}_b']
            idx += 1

    # layer3
    for i in range(3):
        block = model.layer3[i]
        block.conv1.W = weights[f'{idx}_W']
        block.conv1.b = weights[f'{idx}_b']
        idx += 1
        block.conv2.W = weights[f'{idx}_W']
        block.conv2.b = weights[f'{idx}_b']
        idx += 1
        if not block.equal_in_out:
            block.shortcut.W = weights[f'{idx}_W']
            block.shortcut.b = weights[f'{idx}_b']
            idx += 1

    # fc
    model.fc.W = weights['fc_W']
    model.fc.b = weights['fc_b']



## 모델 출력 구하기(test data에 대한  예측)

In [9]:
model = ResNet20() 
load_weights(model, data['model'])

In [10]:
test_logits = model.predict(x_test)
softmax_output = softmax(test_logits) 

##valid에 대해서 하기
test_logits = model.predict(x_val)
softmax_output = softmax(test_logits) 

In [77]:
acc_test_fine = accuracy(softmax_output, y_test)
print(f"Test fine accuracy: {acc_test_fine:.4f}")

Test fine accuracy: 0.0153


## 매핑

In [65]:
##1.Argmx
# 매핑 딕셔너리 불러오기
fine_to_coarse = get_cifar100_fine_to_coarse_dict()

# fine label 예측 → coarse label 매핑
y_fine = np.argmax(softmax_output, axis=1)
y_coarse_argmax = np.array([fine_to_coarse[f] for f in y_fine]) #이게 필요한건가?

# --- 예시 출력 ---
for a in range(30):
    print(f"[{a}] predict fine label:   {y_fine[a]}")
    print(f"    true fine label:      {y_test[a]}")
    print(f"    predict coarse label: {y_coarse_argmx[a]}")
    print(f"    true coarse label:    {y_test_coarse[a]}\n")

[0] predict fine label:   60
    true fine label:      49
    predict coarse label: 10
    true coarse label:    10

[1] predict fine label:   60
    true fine label:      33
    predict coarse label: 10
    true coarse label:    10

[2] predict fine label:   60
    true fine label:      72
    predict coarse label: 10
    true coarse label:    0

[3] predict fine label:   60
    true fine label:      51
    predict coarse label: 10
    true coarse label:    4

[4] predict fine label:   60
    true fine label:      71
    predict coarse label: 10
    true coarse label:    10

[5] predict fine label:   39
    true fine label:      92
    predict coarse label: 5
    true coarse label:    2

[6] predict fine label:   39
    true fine label:      15
    predict coarse label: 5
    true coarse label:    11

[7] predict fine label:   60
    true fine label:      14
    predict coarse label: 10
    true coarse label:    7

[8] predict fine label:   60
    true fine label:      23
    predict 

In [74]:
##2.Entropy weigth
# coarse softmax 분포 계산
coarse_probs = entropy_based_coarse(softmax_output, fine_to_coarse)

# coarse 예측 (argmax)
y_fine = np.argmax(softmax_output, axis=1)
y_coarse = np.argmax(coarse_probs, axis=1)
#얘는 매핑할때 softmax 말고 다른 값이 필요한건가?

# 정답 coarse label
y_coarse_entropy = np.array([fine_to_coarse[f] for f in y_fine])

# --- 예시 출력 ---
for a in range(30):
    print(f"[{a}] predict fine label:   {y_fine[a]}")
    print(f"    true fine label:      {y_test[a]}")
    print(f"    predict coarse label: {y_coarse_entropy[a]}")
    print(f"    true coarse label:    {y_test_coarse[a]}\n")

[0] predict fine label:   60
    true fine label:      49
    predict coarse label: 10
    true coarse label:    10

[1] predict fine label:   60
    true fine label:      33
    predict coarse label: 10
    true coarse label:    10

[2] predict fine label:   60
    true fine label:      72
    predict coarse label: 10
    true coarse label:    0

[3] predict fine label:   60
    true fine label:      51
    predict coarse label: 10
    true coarse label:    4

[4] predict fine label:   60
    true fine label:      71
    predict coarse label: 10
    true coarse label:    10

[5] predict fine label:   39
    true fine label:      92
    predict coarse label: 5
    true coarse label:    2

[6] predict fine label:   39
    true fine label:      15
    predict coarse label: 5
    true coarse label:    11

[7] predict fine label:   60
    true fine label:      14
    predict coarse label: 10
    true coarse label:    7

[8] predict fine label:   60
    true fine label:      23
    predict 

In [67]:
##3.soft label
#soft-label 기반 coarse 분포 계산
coarse_probs = softlabel_coarse_mapping(softmax_output, fine_to_coarse)
#coarse 예측
y_coarse_softlabel = np.argmax(coarse_probs, axis=1)


#acc_coarse = accuracy(coarse_probs, y_test_coarse)
#print(f"Soft-label 기반 coarse acc: {acc_coarse:.4f}")

for a in range(30):
    print(f"[{a}] predict fine label:   {y_fine[a]}")
    print(f"     true fine label:      {y_test[a]}")
    print(f"     predict coarse label: {y_coarse_softlabel[a]}")
    print(f"     true coarse label:    {y_test_coarse[a]}\n")

[0] predict fine label:   60
     true fine label:      49
     predict coarse label: 10
     true coarse label:    10

[1] predict fine label:   60
     true fine label:      33
     predict coarse label: 10
     true coarse label:    10

[2] predict fine label:   60
     true fine label:      72
     predict coarse label: 10
     true coarse label:    0

[3] predict fine label:   60
     true fine label:      51
     predict coarse label: 10
     true coarse label:    4

[4] predict fine label:   60
     true fine label:      71
     predict coarse label: 10
     true coarse label:    10

[5] predict fine label:   39
     true fine label:      92
     predict coarse label: 5
     true coarse label:    2

[6] predict fine label:   39
     true fine label:      15
     predict coarse label: 5
     true coarse label:    11

[7] predict fine label:   60
     true fine label:      14
     predict coarse label: 10
     true coarse label:    7

[8] predict fine label:   60
     true fine la

In [68]:
##4.validation weight
#Step 1. fine-class별 validation 정확도 계산 → fine_weight[100]
#Step 2. softmax_output × fine_weight → weighted_softmax
#Step 3. fine → coarse 그룹 합산 → coarse_probs_weighted
#Step 4. argmax(coarse_probs_weighted) → coarse 예측

In [69]:
# 1. fine accuracy 기반 weight 구하기
fine_acc_weight = compute_fine_class_accuracy(y_fine, y_test)#여기 변수명 점검
# 2. 가중 coarse 확률 분포 만들기
coarse_probs_val_weight = validation_weighted_coarse(softmax_output, fine_acc_weight, fine_to_coarse)
# 3. 예측
y_coarse_valweight = np.argmax(coarse_probs_val_weight, axis=1)

# --- 예시 출력 ---
for a in range(30):
    print(f"[{a}] predict fine label:   {y_fine[a]}")
    print(f"    true fine label:      {y_test[a]}")
    print(f"    predict coarse label: {y_coarse_valweight[a]}")
    print(f"    true coarse label:    {y_test_coarse[a]}\n")

[0] predict fine label:   60
    true fine label:      49
    predict coarse label: 10
    true coarse label:    10

[1] predict fine label:   60
    true fine label:      33
    predict coarse label: 10
    true coarse label:    10

[2] predict fine label:   60
    true fine label:      72
    predict coarse label: 10
    true coarse label:    0

[3] predict fine label:   60
    true fine label:      51
    predict coarse label: 10
    true coarse label:    4

[4] predict fine label:   60
    true fine label:      71
    predict coarse label: 10
    true coarse label:    10

[5] predict fine label:   39
    true fine label:      92
    predict coarse label: 5
    true coarse label:    2

[6] predict fine label:   39
    true fine label:      15
    predict coarse label: 5
    true coarse label:    11

[7] predict fine label:   60
    true fine label:      14
    predict coarse label: 10
    true coarse label:    7

[8] predict fine label:   60
    true fine label:      23
    predict 

## 매핑 기법 정확도 

In [78]:
#정답
y_true_coarse = y_test_coarse

#정확도 계산
acc_coarse_argmax = accuracy(y_coarse_argmax, y_true_coarse)
acc_coarse_entropy = accuracy(y_coarse_entropy, y_true_coarse)
acc_coarse_softlabel = accuracy(y_coarse_softlabel, y_true_coarse)
acc_coarse_valweight = accuracy(y_coarse_valweight, y_true_coarse)


print("📊 Coarse Accuracy 비교")
print(f"1. Argmax 기반         : {acc_coarse_argmax:.5f}")
print(f"2. Entropy-weighted    : {acc_coarse_entropy:.5f}")
print(f"3. Soft-label 평균 기반: {acc_coarse_softlabel:.5f}")
print(f"4. Validation-weight 기반: {acc_coarse_valweight:.5f}")

📊 Coarse Accuracy 비교
1. Argmax 기반         : 0.06420
2. Entropy-weighted    : 0.06420
3. Soft-label 평균 기반: 0.06420
4. Validation-weight 기반: 0.06420


## 함수

### 0.정확도 함수

In [72]:
#라벨 형태, softmax 형태 둘다 가능
def accuracy(pred_or_probs, y_true):
    if pred_or_probs.ndim == 1:
        pred = pred_or_probs  # 라벨 정확도
    else:
        pred = np.argmax(pred_or_probs, axis=1)  # softmax의 정확도
    return np.mean(pred == y_true)

### 1.단순 매핑 함수

In [12]:
##1. 단순 매핑
def load_fine_to_coarse_from_train(data_dir='./cifar-100-python'):
    with open(f"{data_dir}/train", 'rb') as f:
        train_data = pickle.load(f, encoding='bytes')

    fine_labels = train_data[b'fine_labels']
    coarse_labels = train_data[b'coarse_labels']

    fine_to_coarse = {}
    for f, c in zip(fine_labels, coarse_labels):
        if f not in fine_to_coarse:
            fine_to_coarse[f] = c
        else:
            assert fine_to_coarse[f] == c, f"Conflict: {f} → {fine_to_coarse[f]} vs {c}"
    
    return fine_to_coarse



#매핑 테이블
def get_cifar100_fine_to_coarse_dict():
    """
    CIFAR-100의 fine label (0~99)을 coarse label (0~19)로 매핑하는 딕셔너리를 반환
    """
    fine_to_coarse = {
     0:  4,  1:  1,  2: 14,  3:  8,  4:  0,
     5:  6,  6:  7,  7:  7,  8: 18,  9:  3,
    10:  3, 11: 14, 12:  9, 13: 18, 14:  7,
    15: 11, 16:  3, 17:  9, 18:  7, 19: 11,
    20:  6, 21: 11, 22:  5, 23: 10, 24:  7,
    25:  6, 26: 13, 27: 15, 28:  3, 29: 15,
    30:  0, 31: 11, 32:  1, 33: 10, 34: 12,
    35: 14, 36: 16, 37:  9, 38: 11, 39:  5,
    40:  5, 41: 19, 42:  8, 43:  8, 44: 15,
    45: 13, 46: 14, 47: 17, 48: 18, 49: 10,
    50: 16, 51:  4, 52: 17, 53:  4, 54:  2,
    55:  0, 56: 17, 57:  4, 58: 18, 59: 17,
    60: 10, 61:  3, 62:  2, 63: 12, 64: 12,
    65: 16, 66: 12, 67:  1, 68:  9, 69: 19,
    70:  2, 71: 10, 72:  0, 73:  1, 74: 16,
    75: 12, 76:  9, 77: 13, 78: 15, 79: 13,
    80: 16, 81: 19, 82:  2, 83:  4, 84:  6,
    85: 19, 86:  5, 87:  5, 88:  8, 89: 19,
    90: 18, 91:  1, 92:  2, 93: 15, 94:  6,
    95:  0, 96: 17, 97:  8, 98: 14, 99: 13
}
    return fine_to_coarse


def map_softmax_to_coarse(softmax_output, fine_to_coarse):
    """
    softmax_output: shape (N, 100), softmax 확률 결과
    fine_to_coarse: dict, fine label → coarse label 매핑
    
    return: pred_coarse: shape (N,), coarse label 예측 결과
    """
    fine_pred = np.argmax(softmax_output, axis=1)  # shape (N,)
    coarse_pred = np.array([fine_to_coarse[f] for f in fine_pred])  # shape (N,)
    return coarse_pred


### 2.entropy weighted

In [41]:
def entropy_based_coarse(softmax_output, fine_to_coarse, num_coarse=20):
    """
    softmax 결과의 엔트로피를 기반으로 신뢰도를 추정하고,
    coarse-class 확률로 변환하는 후처리 함수입니다.

    Parameters:
    -----------
    softmax_output : np.ndarray
        shape (N, 100) — 모델의 fine-class softmax 출력
    fine_to_coarse : dict
        fine-class index (0~99) → coarse-class index (0~19) 매핑 딕셔너리
    num_coarse : int
        coarse 클래스 개수 (기본: 20)

    Returns:
    --------
    coarse_probs : np.ndarray
        shape (N, 20) — 각 sample의 coarse-class 확률 분포
    """

    N = softmax_output.shape[0]  # sample 개수
    log100 = np.log(100)         # 최대 엔트로피 값 (정규화용)
    coarse_probs = np.zeros((N, num_coarse))  # 결과 coarse 확률 초기화

    for i in range(N):
        # ① 해당 샘플의 softmax 벡터 (fine-class 확률)
        fine_probs = softmax_output[i]  # shape (100,)

        # ② entropy 계산 (불확실성 측정)
        entropy = -np.sum(fine_probs * np.log(fine_probs + 1e-9))  # log(0) 방지

        # ③ 신뢰도 계산: 확신이 높을수록 1에 가까움
        confidence = 1 - (entropy / log100)

        # ④ fine-class 확률에 confidence 곱하기 (확신도 기반 가중)
        weighted_fine_probs = fine_probs * confidence  # shape (100,)

        # ⑤ coarse-class 단위로 합산
        for fine_idx, prob in enumerate(weighted_fine_probs):
            coarse_idx = fine_to_coarse[fine_idx]
            coarse_probs[i][coarse_idx] += prob

    return coarse_probs  # shape (N, 20)

### 3. confidence 기반

In [25]:
def softlabel_coarse_mapping(fine_probs, fine_to_coarse_dict, num_coarse=20):
    batch_size = fine_probs.shape[0]
    coarse_probs = np.zeros((batch_size, num_coarse))

    for fine_idx in range(100):
        coarse_idx = fine_to_coarse_dict[fine_idx]
        coarse_probs[:, coarse_idx] += fine_probs[:, fine_idx]
    
    return coarse_probs

### validation weight

In [48]:
def compute_fine_class_accuracy(pred_fine, true_fine, num_classes=100):
    correct = np.zeros(num_classes)
    total = np.zeros(num_classes)

    for pf, tf in zip(pred_fine, true_fine):
        total[tf] += 1
        if pf == tf:
            correct[tf] += 1

    acc = correct / (total + 1e-9)  # avoid divide by 0
    acc = acc / np.max(acc)        # normalize to [0, 1]
    return acc  # shape (100,)


In [49]:
def validation_weighted_coarse(softmax_output, fine_weight, fine_to_coarse, num_coarse=20):
    weighted_fine = softmax_output * fine_weight[np.newaxis, :]  # shape (N, 100)
    coarse_probs = np.zeros((softmax_output.shape[0], num_coarse))
    
    for f in range(100):
        c = fine_to_coarse[f]
        coarse_probs[:, c] += weighted_fine[:, f]
    
    return coarse_probs  # shape (N, 20)
